# RNNs: predicting Google stock price

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os

Load our data

In [ ]:
dataset_train = pd.read_csv("../input/trainset.csv")
dataset_train.head()

In [ ]:
dataset_train.Date.max()

We'll by trying to predict Google's stock price at opening using windows of 60 past days.

In [ ]:
trainset = dataset_train.iloc[:,1:2].values
trainset

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_scaled = sc.fit_transform(trainset)

In [ ]:
training_scaled

In [ ]:
x_train = []
y_train = []

In [ ]:
window_size = 60

for i in range(window_size,1259):
    x_train.append(training_scaled[i-window_size:i, 0])
    y_train.append(training_scaled[i,0])
    
x_train,y_train = np.array(x_train),np.array(y_train)

In [ ]:
x_train.shape

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

In [ ]:
regressor.compile(optimizer = 'adam',loss = 'mean_squared_error')

In [ ]:
regressor.summary()

In [ ]:
regressor.fit(x_train,
              y_train,
              epochs=30, 
              batch_size=32)

In [ ]:
dataset_test = pd.read_csv("../input/testset.csv")

In [ ]:
dataset_test.head()

In [ ]:
real_stock_price = dataset_test.iloc[:,1:2].values

In [ ]:
dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']),axis = 0)

# Keep only values that start 60 days before dataset_test starts s.t. we can predict with 60 days history
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values

len(inputs) == len(x_test) + window_size

In [ ]:
inputs = inputs.reshape(-1,1)

In [ ]:
# Scale inputs
inputs = sc.transform(inputs)
inputs.shape

In [ ]:
x_test = []
for i in range(60,185):
    x_test.append(inputs[i-60:i,0])

In [ ]:
x_test = np.array(x_test)
x_test.shape

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
x_test.shape

In [ ]:
predicted_price = regressor.predict(x_test)

In [ ]:
predicted_price = sc.inverse_transform(predicted_price)
predicted_price

In [ ]:
plt.plot(real_stock_price,color = 'red', label = 'Real Price')
plt.plot(predicted_price, color = 'blue', label = 'Predicted Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()